In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

dataframe = pd.read_csv("raw results.csv")
dataframe.head()

,ID,dataset,model,noise,noise-args,robust-method,loss-args,eval_loss,eval_accuracy,epochs,batch_size,best-hyperparameters,seed
0,LNSD-341,diabetes,simple-mlp,uniform,0.00,boot-soft,[],0.052478,1.000000,10.0,32.0,"{'num_layers': 3, 'hidden_size_0': 37, 'hidden...",147.0
1,LNSD-1006,breast-cancer,simple-mlp,uniform,0.00,boot-hard,[],0.110205,0.991228,10.0,32.0,"{'num_layers': 1, 'hidden_size_0': 33, 'dropou...",321.0
2,LNSD-910,breast-cancer,simple-mlp,uniform,0.15,none,[],0.219819,0.991228,10.0,32.0,"{'num_layers': 3, 'hidden_size_0': 127, 'hidde...",1746.0
3,LNSD-390,breast-cancer,simple-mlp,uniform,0.15,none,[],0.244681,0.991228,10.0,32.0,"{'num_layers': 3, 'hidden_size_0': 8, 'hidden_...",1147.0
4,LNSD-388,breast-cancer,simple-mlp,uniform,0.10,forward,"[[[0.9, 0.1], [0.1, 0.9]]]",0.161186,0.991228,10.0,32.0,"{'num_layers': 2, 'hidden_size_0': 54, 'hidden...",1147.0


In [102]:
datasets = dataframe["dataset"].unique()
robust_methods = dataframe["robust-method"].unique()
noise_rates = dataframe["noise-args"].unique()
noise_rates.sort()

In [103]:
grouped_multiple = dataframe.groupby(['dataset', 'model', 'noise', 'noise-args', 'robust-method', 'loss-args']).agg({'eval_accuracy': ['mean', 'min', 'max', 'std'], 'eval_loss': ['mean', 'min', 'max', 'std']})
grouped_multiple.columns = ['acc_mean', 'acc_min', 'acc_max', 'acc_std', 'loss_mean', 'loss_min', 'loss_max', 'loss_std']
grouped_multiple = grouped_multiple.reset_index()
print(grouped_multiple)

           dataset       model    noise  noise-args robust-method  \
0    breast-cancer  simple-mlp  uniform         0.0      backward   
1    breast-cancer  simple-mlp  uniform         0.0     boot-hard   
2    breast-cancer  simple-mlp  uniform         0.0     boot-soft   
3    breast-cancer  simple-mlp  uniform         0.0       forward   
4    breast-cancer  simple-mlp  uniform         0.0          none   
..             ...         ...      ...         ...           ...   
100         german  simple-mlp  uniform         0.3      backward   
101         german  simple-mlp  uniform         0.3     boot-hard   
102         german  simple-mlp  uniform         0.3     boot-soft   
103         german  simple-mlp  uniform         0.3       forward   
104         german  simple-mlp  uniform         0.3          none   

                      loss-args  acc_mean   acc_min   acc_max   acc_std  \
0    [[[1.0, 0.0], [0.0, 1.0]]]  0.970395  0.956140  0.982456  0.011425   
1                    

In [104]:
def get_metric_by_dataset(dataset, metric):

    result_mean = np.zeros( (len(robust_methods), len(noise_rates)) )
    result_std = np.zeros( (len(robust_methods), len(noise_rates)) )
    for i, robust_method in enumerate(robust_methods):
        for j, noise_rate in enumerate(noise_rates):
            robust_method_index = grouped_multiple["robust-method"] == robust_method
            noise_rate_index = grouped_multiple["noise-args"] == noise_rate
            dataset_index = grouped_multiple["dataset"] == dataset
            df_index = robust_method_index & noise_rate_index & dataset_index
            result_mean[i,j] = grouped_multiple[df_index]["%s_mean" % metric]
            result_std[i,j] = grouped_multiple[df_index]["%s_std" % metric]

    return result_mean, result_std

In [105]:
def plot_by_dataset(result_mean, result_std, dataset, metric):
    plt.style.use('seaborn-whitegrid')
    fig = plt.figure()
    ax = plt.axes()
    for i, robust_method in enumerate(robust_methods):
        y_err = result_std[i,:]
        ax.plot(noise_rates, result_mean[i,:], label=robust_method)
        ax.fill_between(noise_rates, result_mean[i,:] - y_err, result_mean[i,:] + y_err, alpha=0.2)
    ax.set_title("%s on %s dataset" % (metric, dataset))
    ax.set_xlabel("noise rates")
    ax.set_ylabel(metric);
    ax.legend()

    fig.show()
    fig.savefig("%s %s.png" % (dataset, metric), dpi=300)
    plt.close(fig)

In [106]:
for d in datasets:
    result_mean, result_std =  get_metric_by_dataset(d, 'acc')
    plot_by_dataset(result_mean, result_std, d, 'Test accuracy')

    result_mean, result_std = get_metric_by_dataset(d, 'loss')
    plot_by_dataset(result_mean, result_std, d, 'Test loss')

<ipython-input-105-0db0b8a39139>:14: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()
<ipython-input-105-0db0b8a39139>:14: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()
<ipython-input-105-0db0b8a39139>:14: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()
<ipython-input-105-0db0b8a39139>:14: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()
<ipython-input-105-0db0b8a39139>:14: UserWarning: Matplotlib is currently using module://ipykernel.pylab.backend_inline, which is a non-GUI backend, so cannot show the figure.
  fig.show()
<ipython-input-105-0db0b8a39139>:14: UserWarning: Matpl